In [ ]:
import uproot
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import awkward as ak
import selection
from helpers import *
%load_ext autoreload

In [ ]:
branches = [
    "slc.vertex.x",
    "slc.vertex.y",
    "slc.vertex.z",
    "slc.truth.index",
    "slc.tmatch.pur",
    "slc.tmatch.eff",
    "slc.truth.iscc",
    "slc.truth.pdg",
    "slc.truth.E",
    "slc.truth.position.x",
    "slc.truth.position.y",
    "slc.truth.position.z",
    "slc.truth.prim.length",
    "slc.truth.prim.pdg",
    "slc.truth.nprim",
    "slc.self",
    "slc.nu_score",
    "slc.fmatch.score",
    "slc.fmatch.time",
    "slc.fmatch_a.score",
    "slc.is_clear_cosmic",
    "reco.ntrk",
    "reco.nshw",
    
    "slc.reco.trk.len",
    "slc.reco.trk.costh",
    "slc.reco.trk.phi",
    "slc.reco.trk.start.x",
    "slc.reco.trk.start.y",
    "slc.reco.trk.start.z",
    "slc.reco.trk.end.x",
    "slc.reco.trk.end.y",
    "slc.reco.trk.end.z",
    "slc.reco.trk.ID",
    "slc.reco.trk.slcID",
    "slc.reco.trk.parent_is_primary",
    "slc.reco.trk.chi2pid0.chi2_muon",
    "slc.reco.trk.chi2pid0.chi2_proton",
    "slc.reco.trk.chi2pid1.chi2_muon",
    "slc.reco.trk.chi2pid1.chi2_proton",
    "slc.reco.trk.chi2pid2.chi2_muon",
    "slc.reco.trk.chi2pid2.chi2_proton",
    "slc.reco.trk.bestplane",
    "slc.reco.trk.mcsP.fwdP_muon",
    "slc.reco.trk.rangeP.p_muon",
    "slc.reco.trk.rangeP.p_proton",
    "slc.reco.trk.crthit.hit.time",
    "slc.reco.trk.crthit.distance",
    "slc.reco.trk.crttrack.angle",
    "slc.reco.trk.crttrack.time",
    "slc.reco.trk.truth.bestmatch.G4ID",
    "slc.reco.trk.truth.bestmatch.energy",
    "slc.reco.trk.truth.p.pdg",
    "slc.reco.trk.truth.p.planeVisE",
    "slc.reco.trk.truth.p.gen.x",
    "slc.reco.trk.truth.p.gen.y",
    "slc.reco.trk.truth.p.gen.z",
    "slc.reco.trk.truth.p.genp.x",
    "slc.reco.trk.truth.p.genp.y",
    "slc.reco.trk.truth.p.genp.z",
    "slc.reco.trk.truth.p.startT",
    "slc.reco.trk.truth.p.end.x",
    "slc.reco.trk.truth.p.end.y",
    "slc.reco.trk.truth.p.end.z",
    "slc.reco.trk.truth.p.length",
    "slc.reco.trk.truth.p.contained",
    "slc.reco.trk.truth.p.wallout",
    "slc.reco.trk.truth.p.end_process",
    "slc.reco.trk.truth.total_deposited_energy",
    "mc.nnu",
    "mc.nu.E",
    "mc.nu.pdg",
    "mc.nu.iscc",
    "mc.nu.position.x",
    "mc.nu.position.y",
    "mc.nu.position.z",
    "mc.nu.nprim",
    "mc.nu.prim.pdg",
    "mc.nu.prim.genp.x",
    "mc.nu.prim.genp.y",
    "mc.nu.prim.genp.z",
    "mc.nu.prim.start.x",
    
    
    "true_particles.G4ID",
    "true_particles.pdg",
    "true_particles.genp.x",
    "true_particles.genp.y",
    "true_particles.genp.z",
    "true_particles.interaction_id",
    "true_particles.start.x",
    
    "pass_flashtrig",
    "crt_hits.time",
    "crt_hits.pe",
    "ncrt_hits",
    "crt_tracks.hita.position.y",
    "crt_tracks.hitb.position.y",
    "crt_tracks.time",
    "ncrt_tracks",
    
    "nslc",
    "slc.reco.ntrk",
    "hdr.pot",
    "hdr.evt",
    "hdr.fno",
    "hdr.subrun",
    "hdr.run",
    "hdr.ngenevt",
    "hdr.evt"
]

In [ ]:
treenames = [
    "rec",
    "rec.slc",
    "rec.slc.reco.trk",
    "rec.true_particles",
    "rec.crt_hits",
    "rec.crt_tracks",
    "rec.mc.nu",
    "rec.mc.nu.prim",
    "rec.reco.trk",
    "rec.reco.trk.chi2pid",
    "rec.reco.trk.truth.matches",
    "rec.slc.truth.prim"
]

In [ ]:
do_save = True
savedir = "./plots/"

mpl.rcParams['figure.figsize'] = [6, 4]
mpl.rcParams['figure.dpi'] = 200

In [ ]:
fname = "/pnfs/sbn/persistent/users/gputnam/MCP2020A-CAF/sbnd-overlay.flat.root"
rootf = uproot.open(fname)

data = {}
for b in branches:
    keyname = "rec." + b # ".".join(b.split(".")[1:])
    for t in treenames:
        if not keyname.startswith(t):
            continue
        try:
            d = rootf["recTree"][t].array(keyname)
        except KeyError:
            continue
        data[b] = d
        break
    else:
        raise KeyError(keyname)

groupings = ["slc.reco.trk", "crt_hits", "mc.nu.prim", "slc.truth.prim", "crt_tracks"]
for k in data.keys():
    for g in groupings:
        if k.startswith(g):
            data[k] = group(data[k], data[g.replace(g.split(".")[-1], "n"+g.split(".")[-1])])

to_broadcast = ["pass_flashtrig"]
broadcast_over = "nslc"
for k in to_broadcast:
    data[k] = broadcast(data[k], data[broadcast_over])
    
# save as ovrl 
ovrl = data
del data

In [ ]:
is_nu = ovrl["slc.truth.index"] >= 0
is_cosmic = np.invert(is_nu)
is_numu_cc = is_nu & ovrl["slc.truth.iscc"] & (np.abs(ovrl["slc.truth.pdg"]) == 14)

ovrl["slc.numu_cc"] = is_nu & ovrl["slc.truth.iscc"] & (np.abs(ovrl["slc.truth.pdg"]) == 14)
ovrl["slc.cosmic"] = np.invert(is_nu)

def dist(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

is_fid_true = selection.InFV(ovrl["slc.truth.position.x"], ovrl["slc.truth.position.y"], ovrl["slc.truth.position.z"])
is_fid_reco = selection.InFV(ovrl["slc.vertex.x"], ovrl["slc.vertex.y"], ovrl["slc.vertex.z"])

ovrl["slc.reco.trk.contained"] = selection.InFV(ovrl["slc.reco.trk.end.x"], ovrl["slc.reco.trk.end.y"], ovrl["slc.reco.trk.end.z"])
ovrl["slc.reco.trk.atslc"] = dist(ovrl["slc.reco.trk.start.x"], ovrl["slc.reco.trk.start.y"], ovrl["slc.reco.trk.start.z"],
                                 ovrl["slc.vertex.x"], ovrl["slc.vertex.y"], ovrl["slc.vertex.z"]) < 10

ovrl["slc.reco.trk.truth.p.is_stopping"] = (ovrl["slc.reco.trk.truth.p.end_process"] == 1) |\
                                    (ovrl["slc.reco.trk.truth.p.end_process"] == 2) |\
                                    (ovrl["slc.reco.trk.truth.p.end_process"] == 3) |\
                                    (ovrl["slc.reco.trk.truth.p.end_process"] == 41)

ovrl["slc.reco.trk.recop"] = ovrl["slc.reco.trk.rangeP.p_muon"] + 0. # clone
ovrl["slc.reco.trk.recop"][np.invert(ovrl["slc.reco.trk.contained"])] = ovrl["slc.reco.trk.mcsP.fwdP_muon"][np.invert(ovrl["slc.reco.trk.contained"])] 

for chi2 in ["chi2_muon", "chi2_proton"]:
    ovrl["slc.reco.trk.bestplane." + chi2] = ovrl["slc.reco.trk.chi2pid2." + chi2]
    ovrl["slc.reco.trk.bestplane." + chi2][ovrl["slc.reco.trk.bestplane"] == 0] = ovrl["slc.reco.trk.chi2pid0." + chi2][ovrl["slc.reco.trk.bestplane"] == 0]
    ovrl["slc.reco.trk.bestplane." + chi2][ovrl["slc.reco.trk.bestplane"] == 1] = ovrl["slc.reco.trk.chi2pid1." + chi2][ovrl["slc.reco.trk.bestplane"] == 1]

# For the cases where two slices match to the same neutrino, figure out which match is "primary"
# Use whichever slice gets more of the deposited energy
ovrl["slc.truth.match_is_primary"] = ovrl["slc.truth.iscc"] & False # clone
# Check the max match for each neutrino match 
# Group each slice by spill
index_spill = group(ovrl["slc.truth.index"], ovrl["nslc"])
eff_spill = group(ovrl["slc.tmatch.eff"], ovrl["nslc"])
primary_spill = group(ovrl["slc.truth.match_is_primary"], ovrl["nslc"])

for i in range(ovrl["slc.truth.index"].max()+1): # consider up to the maximum nuetrinos
    primary_spill = primary_spill | (eff_spill[index_spill==i].max() == eff_spill) & (eff_spill[index_spill==i].max() > 0.)

ovrl["slc.truth.match_is_primary"] = primary_spill.flatten()
# all cosmic matches are primary
ovrl["slc.truth.match_is_primary"] = ovrl["slc.truth.match_is_primary"] | (ovrl["slc.truth.index"] < 0)

In [ ]:
# Verify that the truth matching worked as expected

# bad 
for i in range(ovrl["slc.truth.index"].max()+1):
    bad = ((index_spill == i) & primary_spill).sum() > 1
    assert(not bad.any())
    
# also bad
for i in range(ovrl["slc.truth.index"].max()+1):
    bad = ((index_spill == i).sum() > 0) & (np.invert(ovrl["slc.truth.match_is_primary"].any()))
    assert(not bad.any())

when = ((primary_spill == False) & (index_spill >= 0)).any()

#print(eff_spill[when][0])
#print(index_spill[when][0])
#print(primary_spill[when][0])

In [ ]:
fname = "/pnfs/sbn/persistent/users/gputnam/MCP2020A-CAF/sbnd-intime.flat.root"
rootf = uproot.open(fname)

data = {}
for b in branches:
    keyname = "rec." + b # ".".join(b.split(".")[1:])
    for t in treenames:
        if not keyname.startswith(t):
            continue
        try:
            d = rootf["recTree"][t].array(keyname)
        except KeyError:
            continue
        data[b] = d
        break
    else:
        raise KeyError(keyname)

groupings = ["slc.reco.trk", "crt_hits", "mc.nu.prim", "slc.truth.prim", "crt_tracks"]
for k in data.keys():
    for g in groupings:
        if k.startswith(g):
            data[k] = group(data[k], data[g.replace(g.split(".")[-1], "n"+g.split(".")[-1])])

to_broadcast = ["pass_flashtrig"]
broadcast_over = "nslc"
for k in to_broadcast:
    data[k] = broadcast(data[k], data[broadcast_over])
    
# save as intc 
intc = data
del data

In [ ]:
intc["slc.numu_cc"] = intc["slc.truth.iscc"] # always False
intc["slc.cosmic"] = np.invert(intc["slc.truth.iscc"]) # always True
intc["slc.truth.match_is_primary"] = intc["slc.cosmic"] # always True

intc["slc.reco.trk.contained"] = selection.InFV(intc["slc.reco.trk.end.x"], intc["slc.reco.trk.end.y"], intc["slc.reco.trk.end.z"])
intc["slc.reco.trk.atslc"] = dist(intc["slc.reco.trk.start.x"], intc["slc.reco.trk.start.y"], intc["slc.reco.trk.start.z"],
                                 intc["slc.vertex.x"], intc["slc.vertex.y"], intc["slc.vertex.z"]) < 10

intc["slc.reco.trk.truth.p.is_stopping"] = (intc["slc.reco.trk.truth.p.end_process"] == 1) |\
                                    (intc["slc.reco.trk.truth.p.end_process"] == 2) |\
                                    (intc["slc.reco.trk.truth.p.end_process"] == 3) |\
                                    (intc["slc.reco.trk.truth.p.end_process"] == 41)

intc["slc.reco.trk.recop"] = intc["slc.reco.trk.rangeP.p_muon"] + 0. # clone
intc["slc.reco.trk.recop"][np.invert(intc["slc.reco.trk.contained"])] = intc["slc.reco.trk.mcsP.fwdP_muon"][np.invert(intc["slc.reco.trk.contained"])] 

for chi2 in ["chi2_muon", "chi2_proton"]:
    intc["slc.reco.trk.bestplane." + chi2] = intc["slc.reco.trk.chi2pid2." + chi2]
    intc["slc.reco.trk.bestplane." + chi2][intc["slc.reco.trk.bestplane"] == 0] = intc["slc.reco.trk.chi2pid0." + chi2][intc["slc.reco.trk.bestplane"] == 0]
    intc["slc.reco.trk.bestplane." + chi2][intc["slc.reco.trk.bestplane"] == 1] = intc["slc.reco.trk.chi2pid1." + chi2][intc["slc.reco.trk.bestplane"] == 1]

In [ ]:
ovrl_pot = NeutrinoPOT(ovrl)
intc_pot = CosmicPOT(intc, ovrl)
goal_pot = 6.6e20

ovrl_pot_scale = goal_pot / ovrl_pot
intc_pot_scale = goal_pot / intc_pot

print(ovrl_pot_scale)
print(intc_pot_scale)
print(intc["slc.cosmic"].all())

In [ ]:
primary_track = selection.get_primary_tracks(ovrl)
primary_track_ind = ak.JaggedArray.fromcounts((primary_track >= 0)*1, primary_track[primary_track >= 0])
true_primary_track = selection.get_true_primary_track(ovrl)
true_primary_track_ind = ak.JaggedArray.fromcounts((true_primary_track >= 0)*1, true_primary_track[true_primary_track >= 0])

In [ ]:
# Set all of the primary track slice variables
keys = list(ovrl.keys())
for k in keys:
    if k.startswith("slc.reco.trk."):
        slc_key = k.replace(".reco.trk.", ".ptrk.")
        ovrl[slc_key] = np.empty(ovrl["slc.nu_score"].shape)
        ovrl[slc_key][:] = np.NaN
        
        is_bool = ovrl[k][0].dtype == "bool"
        d = ovrl[k] + 0 # clone
        ovrl[slc_key][primary_track >= 0] = d[primary_track_ind[primary_track_ind >= 0]].flatten()
        if is_bool:
            ovrl[slc_key] = ovrl[slc_key] == 1 # restore bool-type arrays
ovrl["slc.has_ptrk"] = primary_track >= 0

In [ ]:
primary_track = selection.get_primary_tracks(intc)
primary_track_ind = ak.JaggedArray.fromcounts((primary_track >= 0)*1, primary_track[primary_track >= 0])
true_primary_track = selection.get_true_primary_track(intc)
true_primary_track_ind = ak.JaggedArray.fromcounts((true_primary_track >= 0)*1, true_primary_track[true_primary_track >= 0])

In [ ]:
# Set all of the primary track slice variables
keys = list(intc.keys())
for k in keys:
    if k.startswith("slc.reco.trk."):
        slc_key = k.replace(".reco.trk.", ".ptrk.")
        intc[slc_key] = np.empty(intc["slc.nu_score"].shape)
        intc[slc_key][:] = np.NaN
        
        is_bool = intc[k][0].dtype == "bool"
        d = intc[k] + 0 # clone
        intc[slc_key][primary_track >= 0] = d[primary_track_ind[primary_track_ind >= 0]].flatten()
        if is_bool:
            intc[slc_key] = intc[slc_key] == 1 # restore bool-type arrays
intc["slc.has_ptrk"] = primary_track >= 0

In [ ]:
# Define the Cuts!!!!!
fid_ovrl = selection.fid(ovrl)
nu_score_ovrl = selection.nu_score(ovrl)
f_time_ovrl = selection.f_time(ovrl)
f_score_ovrl = selection.f_score(ovrl)
ptrk_ovrl = selection.ptrk(ovrl)
crttrack_ovrl = selection.crttrack(ovrl)
crthit_ovrl = selection.crthit(ovrl)
crttrackveto_ovrl = selection.crttrackveto(ovrl)
crttrackveto_no_bottom_ovrl = selection.crttrackveto_nobottom(ovrl)

In [ ]:
# Define the Cuts!!!!!
fid_intc = selection.fid(intc)
nu_score_intc = selection.nu_score(intc)
f_time_intc = selection.f_time(intc)
f_score_intc = selection.f_score(intc)
ptrk_intc = selection.ptrk(intc)
crttrack_intc = selection.crttrack(intc)
crthit_intc = selection.crthit(intc)
crtveto_intc = selection.crtveto(intc)
crttrackveto_intc = selection.crttrackveto(intc)
crttrackveto_no_bottom_intc = selection.crttrackveto_nobottom(intc)

In [ ]:
def suppression(rate, eff):
    # time: 3 years
    time = 3 * 365 * 24 * 60 * 60
    # number of triggers
    ntrig = time * rate
    ret = np.sqrt(ntrig * eff)
    return ret

def rate_to_enhancement(rate, n_evt):
    # time: 3 years
    time = 3 * 365 * 24 * 60 * 60
    # number of triggers
    ntrig = time * rate
    return ntrig / n_evt

def rate_to_read_time(rate, filtering):
    # time: 3 years
    time = 3 * 365 * 24 * 60 * 60
    # size per event: 37.5MB
    eventsize = 37.5 
    # get the total data volume in TB
    data = rate * time * eventsize / (1024. * 1024.)
    # take out the fraction that will be filtered out
    data = data * filtering
    # read speed: 1TB / hour -> 24TB / day
    read = 24
    return data / read
    
    
def uncertainty_plot(count_funcs, names, data_bkg, scale_bkg, data_sig, scale_sig, n_evt, filtering, xtitle, ytitle, figname, text=None):
    plt.gcf().subplots_adjust(bottom=0.25)
    # rates = np.linspace(0.2,10,99)
    rates = np.logspace(-3, 1, 50)
    for f, n in zip(count_funcs, names):
        contamination = f(data_bkg, scale_bkg) / f(data_sig, scale_sig)
        eff = f(data_bkg, scale_bkg) / n_evt
        plt.plot(rates, contamination / suppression(rates, eff), label=n)
    plt.xlabel(xtitle) # "3-Year Triggered Data Rate [Hz]")
    plt.ylabel(ytitle) #"Intime-Cosmic Fractional Uncertainty")

    plt.xscale("log")
    #plt.yscale("log")
    ax1 = plt.gca()
    ax1.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    # set the xticks to start at the minimum x value
    # ticks, labels = plt.xticks()
    # ticks = [0, 0.2, 2, 4, 6, 8, 10]
    # labels = ["", "0.2", "2", "4", "6", "8", "10"]
    # plt.xticks(ticks, labels)
    
    ax1.legend()
    
    t = ax1.yaxis.get_offset_text()
    t.set_x(-0.1)

    ax1.grid(axis="x")
    ax1.grid(axis="y")
    ax_top = plt.gca().twiny()
    lo, hi = ax1.get_xlim()


    ax_top.set_xlabel("Data Enhacement $\mathregular{N_{off}}/\mathregular{N_{on}}$")
    ax_top.set_xscale("log")
    ax_top.set_xlim(rate_to_enhancement(lo, n_evt), 
                    rate_to_enhancement(hi, n_evt))

    
    # add another x-axis
    ax2 = ax1.twiny()
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    # Offset the twin axis below the host
    ax2.spines["bottom"].set_position(("axes", -0.2))

    # Turn on the frame for the twin axis, but then hide all 
    # but the bottom spine
    ax2.set_frame_on(True)
    ax2.patch.set_visible(False)

    for sp in ax2.spines.values():
        sp.set_visible(False)
    ax2.spines["bottom"].set_visible(True)

    ax2.set_xlim(rate_to_read_time(lo, filtering), rate_to_read_time(hi, filtering))
    ax2.set_xscale("log")
    ax2.set_xlabel("Time to Read Filtered Raw Data Off Tape [days]")
    
    # add any textbox
    if text:
        plt.text(0.15, 0.7, text, transform=ax1.transAxes, fontsize=16)
    
    if figname and outputdir: plt.savefig(outputdir + figname) # "%s_intime_error_SBND_binned.pdf" % rqual)

In [ ]:
when_ovrl = fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl
when_intc = fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc

In [ ]:
def count_func(lo, hi):
    return lambda d, scale: np.sum((d > lo) & (d < hi)) * scale

recops = np.linspace(0.2, 1., 5)
vname = "slc.ptrk.recop"

count_funcs = [count_func(recops[i], recops[i+1]) for i in range(len(recops)-1)]
names = ["%.1f < $P_\mu$ < %.1fGeV" % (recops[i], recops[i+1]) for i in range(len(recops)-1)]


In [ ]:
plt.figure(0)
uncertainty_plot(count_funcs, names, ovrl[vname][when_ovrl & is_cosmic], ovrl_pot_scale,
                 ovrl[vname][when_ovrl & is_numu_cc], ovrl_pot_scale,
                NEvt(ovrl) * ovrl_pot_scale, 1., 
                 "3-Year Un-Triggered Data Rate [Hz]", 
                 "Fractional Uncertainty on Prediction",
                "",
                "Neutrino-Triggered\nCosmic")
if do_save: plt.savefig(savedir + "neutrino_triggered_uncertainty.png")

In [ ]:
plt.figure(0)
uncertainty_plot(count_funcs, names, intc[vname][when_intc], intc_pot_scale,
                 ovrl[vname][when_ovrl & is_numu_cc], ovrl_pot_scale,
                NEvt(intc) * intc_pot_scale, 1., 
                 "3-Year Triggered Data Rate [Hz]", 
                 "Fractional Uncertainty on Prediction",
                "",
                "Cosmic-Triggered\nCosmic")

if do_save: plt.savefig(savedir + "cosmic_triggered_uncertainty.png")

In [ ]:
def spectra_plot(data, when, trackveto, trackveto_no_bottom, scale, name):
    var = data["slc.ptrk.recop"]
    
    _ = plt.hist([var[when], var[when & trackveto], var[when & trackveto_no_bottom]],
            weights=[np.repeat(scale, np.sum(when)),
                  np.repeat(scale, np.sum(when & trackveto)),
                  np.repeat(scale, np.sum(when & trackveto_no_bottom))],
             histtype="step",
             bins=np.linspace(0, 2, 21),
             label=["No Veto", "All Planes Veto", "Bottom Plane Excl. Veto"])
    plt.xlabel("Reconstructed Muon Momentum [GeV]")
    plt.ylabel("Neutrino Candidates / 6.6e20")
    plt.legend()
    plt.gca().ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.text(0.5, 0.4, name, transform=plt.gca().transAxes, fontsize=16)
    

In [ ]:
# Now look at the impact of the CRT track veto
spectra_plot(ovrl, is_numu_cc & when_ovrl, crttrackveto_ovrl, crttrackveto_no_bottom_ovrl, ovrl_pot_scale, "Full Event Selection\n$\\nu_\\mu$ CC")
if do_save: plt.savefig(savedir + "crttrack_veto_spectra_full_numucc.png")

In [ ]:
# Now look at the impact of the CRT track veto
spectra_plot(ovrl, is_numu_cc & fid_ovrl & nu_score_ovrl, crttrackveto_ovrl, crttrackveto_no_bottom_ovrl, ovrl_pot_scale, 
             "TPC Pre-Selection\n$\\nu_\\mu$ CC")
if do_save: plt.savefig(savedir + "crttrack_veto_spectra_tpc_numucc.png")

In [ ]:
# Now look at the impact of the CRT track veto
spectra_plot(ovrl, is_cosmic & when_ovrl, crttrackveto_ovrl, crttrackveto_no_bottom_ovrl, ovrl_pot_scale, 
             "Full Event Selection\nNeutrino-Triggered\nCosmic")
if do_save: plt.savefig(savedir + "crttrack_veto_spectra_full_ovrlcosmic.png")

In [ ]:
# Now look at the impact of the CRT track veto
spectra_plot(ovrl, is_cosmic & fid_ovrl & nu_score_ovrl, crttrackveto_ovrl, crttrackveto_no_bottom_ovrl, ovrl_pot_scale, 
        "TPC Pre-Selection\nNeutrino-Triggered\nCosmic")
if do_save: plt.savefig(savedir + "crttrack_veto_spectra_tpc_ovrlcosmic.png")

In [ ]:
# Now look at the impact of the CRT track veto
spectra_plot(intc, when_intc, crttrackveto_intc, crttrackveto_no_bottom_intc, intc_pot_scale, 
             "Full Event Selection\nCosmic-Triggered\nCosmic")
if do_save: plt.savefig(savedir + "crttrack_veto_spectra_full_intccosmic.png")

In [ ]:
# Now look at the impact of the CRT track veto
spectra_plot(intc, fid_intc & nu_score_intc, crttrackveto_intc, crttrackveto_no_bottom_intc, intc_pot_scale, 
             "TPC Pre-Selection\nCosmic-Triggered\nCosmic")
if do_save: plt.savefig(savedir + "crttrack_veto_spectra_tpc_intccosmic.png")

In [ ]:
crttrackveto_perevt_ovrl = selection.crttrackveto_perevt(ovrl)
crttrackveto_no_bottom_perevt_ovrl = selection.crttrackveto_nobottom_perevt(ovrl)

crttrackveto_perevt_intc = selection.crttrackveto_perevt(intc)
crttrackveto_no_bottom_perevt_intc = selection.crttrackveto_nobottom_perevt(intc)

In [ ]:
# print the main data filtering numbers
print("All Planes Veto & %.1f & %.1f\\\\" % (100 - 100.*np.sum(crttrackveto_perevt_intc)/ len(crttrackveto_perevt_intc),
    100 - 100. *np.sum(crttrackveto_perevt_ovrl)/ len(crttrackveto_perevt_ovrl)))

# print the main data filtering numbers
print("Bottom Plane Excl. Veto & %.1f & %.1f\\\\" % (100 - 100.*np.sum(crttrackveto_no_bottom_perevt_intc)/ len(crttrackveto_no_bottom_perevt_intc),
    100 - 100. *np.sum(crttrackveto_no_bottom_perevt_ovrl)/ len(crttrackveto_no_bottom_perevt_ovrl)))

In [ ]:
plt.figure(0)

uncertainty_plot(count_funcs, names, ovrl[vname][when_ovrl & crttrackveto_ovrl & is_cosmic], ovrl_pot_scale,
                 ovrl[vname][when_ovrl & crttrackveto_ovrl& is_numu_cc], ovrl_pot_scale,
                NEvt(ovrl) * ovrl_pot_scale, np.sum(crttrackveto_perevt_ovrl)/ len(crttrackveto_perevt_ovrl), 
                 "3-Year Un-Triggered Data Rate [Hz]", 
                 "Fractional Uncertainty on Prediction",
                "",
                "Neutrino-Triggered\nCosmic\nAll Plane Veto")
if do_save: plt.savefig(savedir + "neutrino_triggered_uncertainty_allplaneveto.png")

In [ ]:
plt.figure(0)
uncertainty_plot(count_funcs, names, intc[vname][when_intc & crttrackveto_intc], intc_pot_scale,
                 ovrl[vname][when_ovrl & crttrackveto_ovrl & is_numu_cc], ovrl_pot_scale,
                NEvt(intc) * intc_pot_scale, np.sum(crttrackveto_perevt_intc)/ len(crttrackveto_perevt_intc), 
                 "3-Year Triggered Data Rate [Hz]", 
                 "Fractional Uncertainty on Prediction",
                "",
                "Cosmic-Triggered\nCosmic\nAll Plane Veto")

if do_save: plt.savefig(savedir + "cosmic_triggered_uncertainty_allplaneveto.png")

In [ ]:
plt.figure(0)

uncertainty_plot(count_funcs, names, ovrl[vname][when_ovrl & crttrackveto_no_bottom_ovrl & is_cosmic], ovrl_pot_scale,
                 ovrl[vname][when_ovrl & crttrackveto_no_bottom_ovrl& is_numu_cc], ovrl_pot_scale,
                NEvt(ovrl) * ovrl_pot_scale, np.sum(crttrackveto_no_bottom_perevt_ovrl)/ len(crttrackveto_no_bottom_perevt_ovrl), 
                 "3-Year Un-Triggered Data Rate [Hz]", 
                 "Fractional Uncertainty on Prediction",
                "",
                "Neutrino-Triggered\nCosmic\nBottom Plane Excl.")
if do_save: plt.savefig(savedir + "neutrino_triggered_uncertainty_nobottomplaneveto.png")

In [ ]:
plt.figure(0)
uncertainty_plot(count_funcs, names, intc[vname][when_intc & crttrackveto_no_bottom_intc], intc_pot_scale,
                 ovrl[vname][when_ovrl & crttrackveto_no_bottom_ovrl & is_numu_cc], ovrl_pot_scale,
                NEvt(intc) * intc_pot_scale, np.sum(crttrackveto_no_bottom_perevt_intc)/ len(crttrackveto_no_bottom_perevt_intc), 
                 "3-Year Triggered Data Rate [Hz]", 
                 "Fractional Uncertainty on Prediction",
                "",
                "Cosmic-Triggered\nCosmic\nBottom Plane Excl.")

if do_save: plt.savefig(savedir + "cosmic_triggered_uncertainty_nobottomplaneveto.png")